In [42]:
import boto3
import time
import json

session = boto3.Session(profile_name='nikita')

client = session.client('batch')
log_client = session.client('logs')


import numpy as np
import math

In [43]:
def submit_job(a, b, pool, side, timeout):
    return client.submit_job(
        jobDefinition='simulate',
        jobName=pool,
        jobQueue='first-run-job-queue',
        containerOverrides={
            'command': ["python", "simulate.py"],
            'environment': [
                {'name': 'PLAYER_A', 'value': a % 0},
                {'name': 'PLAYER_B', 'value': b % 1},
                {'name': 'POOL','value': pool},
                {'name': 'SIDE', 'value': str(side)},
            ]
        },
        timeout={'attemptDurationSeconds': timeout},
    )

def sumbit_double(a, b, pool, timeout=600):
    submit_job(a, b, pool, side=0, timeout=timeout)
    submit_job(b, a, pool, side=1, timeout=timeout)

## Compare the power of MCTS(5s) vs a wide range of values

In [34]:
space = [0.5, 1, 1.5, 2, 3, 5, 7, 10, 15, 20, 30, 40]

for i in range(5):
    for time in space:
            a = "MCTSPlayer(%s, td(seconds=5))"
            b = f"MCTSPlayer(%s, td(seconds={time}))"

            sumbit_double(a, b, "mcts_5s_time_compare_longer", timeout=60*100)

## Compare UTC(c=sqrt(2)/2) vs a range of other values

In [14]:
space = np.linspace(0, 2, 11) + 0.2
space

array([0.2, 0.4, 0.6, 0.8, 1. , 1.2, 1.4, 1.6, 1.8, 2. , 2.2])

## Compare UTC(c=1.5) vs a range of other values

In [17]:
space = np.linspace(0, 2, 11) + 0.2
print(space)

for i in range(25):
    for c in space:
            a = "UCTPlayer(%s, td(seconds=3), c=1.5)"
            b = f"UCTPlayer(%s, td(seconds=3), c={c:.2f})"

            sumbit_double(a, b, "uct-1_5-vs-range")

print("Done")

[0.2 0.4 0.6 0.8 1.  1.2 1.4 1.6 1.8 2.  2.2]


## Tournament

In [50]:
algos = [
    "RandomPlayer(%i)",
    "GreedyPlayer(%i, 0.9)",
    "MCTSPlayer(%i, td(seconds=5))",
    "UCTPlayer(%i, td(seconds=5), c=math.sqrt(2)/2)",
    "GreedyUCTPlayer(%i, td(seconds=5), c=math.sqrt(2)/2)",
]

for i in range(10):
    for a in algos:
        for b in algos:
            sumbit_double(a, b, "tournament-1")